<a href="https://colab.research.google.com/github/Ronokhasan8781/CNN-Code/blob/main/Gastric_Classification_(InceptionV3_%2B_ResNet50).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

ronokhasan_gastric_path = kagglehub.dataset_download('ronokhasan/gastric')

print('Data source import complete.')


In [ ]:
# Standard data science libraries
import psutil
import humanize
import os
from IPython.display import display_html

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
dataDirectory= "/kaggle/input/gastric"
print(os.listdir(dataDirectory))

# Any results you write to the current directory are saved as output.

In [ ]:
print(os.listdir("../input"))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Activation, BatchNormalization, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Use this import
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from sklearn.metrics import confusion_matrix, average_precision_score, recall_score, precision_score, accuracy_score, classification_report
from tensorflow.keras.models import model_from_json
import itertools
import matplotlib.pyplot as plt
import time
import pandas as pd

# Ensure inline plotting for Jupyter Notebook
%matplotlib inline


In [ ]:
train_path = dataDirectory+'/train'
test_path  = dataDirectory+'/test'
print(os.listdir(train_path))
print(os.listdir(test_path))

In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest',
    validation_split=0.2) # set validation split

In [ ]:
#['Burger King','HD Iskender','Kahve Dunyasi', 'KFC','McDonalds','Other', 'Ozsut','Popeyes',  'Starbucks', 'Subway', 'Tavuk Dunyasi']
selectedClasses = ['0_normal', '2_polyps','3_esophagitis','1_ulcerative_colitis']

In [ ]:
batchSize=32


train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_path, # same directory as training data
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='validation') # set as validation data

test_generator = ImageDataGenerator().flow_from_directory(
    test_path,
    target_size=(224,224),
    classes=selectedClasses,
    shuffle= False,
    batch_size = batchSize)# set as test data

In [ ]:
print ("In train_generator ")
for cls in range(len (train_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(train_generator.classes).count(cls))
print ("")

print ("In validation_generator ")
for cls in range(len (validation_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(validation_generator.classes).count(cls))
print ("")

print ("In test_generator ")
for cls in range(len (test_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(test_generator.classes).count(cls))

In [ ]:
#plots images with labels within jupyter notebook
def plots(ims, figsize = (22,22), rows=4, interp=False, titles=None, maxNum = 9):
    if type(ims[0] is np.ndarray):
        ims = np.array(ims).astype(np.uint8)
        if(ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))

    f = plt.figure(figsize=figsize)
    #cols = len(ims) //rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    cols = maxNum // rows if maxNum % 2 == 0 else maxNum//rows + 1
    #for i in range(len(ims)):
    for i in range(maxNum):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=20)
        plt.imshow(ims[i], interpolation = None if interp else 'none')

In [ ]:
# Standard data science libraries
import psutil
import humanize
import os
from IPython.display import display_html

# Helpful analytics libraries
import numpy as np
import pandas as pd

# Display contents of the input directory
dataDirectory = "/kaggle/input/gastric"
print(os.listdir(dataDirectory))

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Activation, BatchNormalization, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from sklearn.metrics import confusion_matrix, average_precision_score, recall_score, precision_score, accuracy_score, classification_report
from tensorflow.keras.models import model_from_json
import itertools
import matplotlib.pyplot as plt
import time

# Ensure inline plotting for Jupyter Notebook
%matplotlib inline

# Paths for training and testing data
train_path = dataDirectory + '/train'
test_path = dataDirectory + '/test'

print(os.listdir(train_path))
print(os.listdir(test_path))

# ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    validation_split=0.2  # Set validation split
)

# Selected classes for classification
selectedClasses = ['0_normal', '2_polyps', '3_esophagitis', '1_ulcerative_colitis']

# Batch size
batchSize = 32

# Generators for training, validation, and testing data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='validation'
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_path,
    target_size=(224, 224),
    classes=selectedClasses,
    shuffle=False,
    batch_size=batchSize
)

# Count and display class distribution in generators
print("In train_generator")
for cls in range(len(train_generator.class_indices)):
    print(selectedClasses[cls], ":\t", list(train_generator.classes).count(cls))
print("")

print("In validation_generator")
for cls in range(len(validation_generator.class_indices)):
    print(selectedClasses[cls], ":\t", list(validation_generator.classes).count(cls))
print("")

print("In test_generator")
for cls in range(len(test_generator.class_indices)):
    print(selectedClasses[cls], ":\t", list(test_generator.classes).count(cls))

# Function to plot images with labels
def plots(ims, figsize=(22, 22), rows=4, interp=False, titles=None, maxNum=9):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if ims.shape[-1] != 3:
            ims = ims.transpose((0, 2, 3, 1))
    f = plt.figure(figsize=figsize)
    cols = maxNum // rows if maxNum % rows == 0 else maxNum // rows + 1
    for i in range(min(maxNum, len(ims))):
        sp = f.add_subplot(rows, cols, i + 1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=20)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

# Fetch and display the first batch of training images and their labels
train_generator.reset()
imgs, labels = next(train_generator)  # Corrected method to fetch batch

# Decode label indices to class names
labelNames = []
labelIndices = [np.where(r == 1)[0][0] for r in labels]
for ind in labelIndices:
    for labelName, labelIndex in train_generator.class_indices.items():
        if labelIndex == ind:
            labelNames.append(labelName)

# Display images with their labels
plots(imgs, titles=labelNames)


In [ ]:
#InceptionV3

base_model = InceptionV3(weights='imagenet',
                                include_top=False,
                                input_shape=(224, 224,3))
base_model.trainable = False

x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dropout(0.5)(x)
# and a sofymax/logistic layer -- we have 6 classes
predictions = Dense(len(selectedClasses), activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)


model.summary()

In [ ]:
# Standard data science libraries
import psutil
import humanize
import os
from IPython.display import display_html

# Helpful analytics libraries
import numpy as np
import pandas as pd

# Display contents of the input directory
dataDirectory = "/kaggle/input/gastric"
print(os.listdir(dataDirectory))

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Activation, BatchNormalization, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from sklearn.metrics import confusion_matrix, average_precision_score, recall_score, precision_score, accuracy_score, classification_report
from tensorflow.keras.models import model_from_json
import itertools
import matplotlib.pyplot as plt
import time

# Ensure inline plotting for Jupyter Notebook
%matplotlib inline

# Paths for training and testing data
train_path = dataDirectory + '/train'
test_path = dataDirectory + '/test'

print(os.listdir(train_path))
print(os.listdir(test_path))

# ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    validation_split=0.2  # Set validation split
)

# Selected classes for classification
selectedClasses = ['0_normal', '2_polyps', '3_esophagitis', '1_ulcerative_colitis']

# Batch size
batchSize = 32

# Generators for training, validation, and testing data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='validation'
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_path,
    target_size=(224, 224),
    classes=selectedClasses,
    shuffle=False,
    batch_size=batchSize
)

# Count and display class distribution in generators
print("In train_generator")
for cls in range(len(train_generator.class_indices)):
    print(selectedClasses[cls], ":\t", list(train_generator.classes).count(cls))
print("")

print("In validation_generator")
for cls in range(len(validation_generator.class_indices)):
    print(selectedClasses[cls], ":\t", list(validation_generator.classes).count(cls))
print("")

print("In test_generator")
for cls in range(len(test_generator.class_indices)):
    print(selectedClasses[cls], ":\t", list(test_generator.classes).count(cls))

# Function to plot images with labels
def plots(ims, figsize=(22, 22), rows=4, interp=False, titles=None, maxNum=9):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if ims.shape[-1] != 3:
            ims = ims.transpose((0, 2, 3, 1))
    f = plt.figure(figsize=figsize)
    cols = maxNum // rows if maxNum % rows == 0 else maxNum // rows + 1
    for i in range(min(maxNum, len(ims))):
        sp = f.add_subplot(rows, cols, i + 1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=20)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

# Fetch and display the first batch of training images and their labels
train_generator.reset()
imgs, labels = next(train_generator)  # Corrected method to fetch batch

# Decode label indices to class names
labelNames = []
labelIndices = [np.where(r == 1)[0][0] for r in labels]
for ind in labelIndices:
    for labelName, labelIndex in train_generator.class_indices.items():
        if labelIndex == ind:
            labelNames.append(labelName)

# Display images with their labels
plots(imgs, titles=labelNames)

# InceptionV3 Model
base_model = InceptionV3(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout layer for regularization
predictions = Dense(len(selectedClasses), activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

model.summary()


In [ ]:
# Automatic rename with epoch number and validation accuracy:
# filepath = "checkpoints/weights-improvement-epoch-{epoch:02d}-val_acc-{val_acc:.2f}.keras"

# Model name and checkpoint filepath
modelName = "InceptionTutorial"
# Save the best weights with the model name
filepath = modelName + "_bestweights.keras"  # Updated extension to `.keras`

# Define ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath,
    monitor='val_acc',  # Use 'val_accuracy' if using TensorFlow 2.4 or later
    verbose=1,
    save_best_only=True,
    mode='max'
)
callbacks_list = [checkpoint]


In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# Ensure the correct metric is used
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Optimizer with learning rate
    loss='categorical_crossentropy',       # Loss function
    metrics=['accuracy']                   # Metric for training and validation
)

# Updated ModelCheckpoint to monitor 'val_accuracy'
filepath = modelName + "_bestweights.keras"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='val_accuracy',  # Correct validation metric to monitor
    verbose=1,
    save_best_only=True,
    mode='max'
)
callbacks_list = [checkpoint]

# Reset the generators (if necessary)
train_generator.reset()
validation_generator.reset()

# Ensure sufficient data is generated for steps_per_epoch and validation_steps
if stepsPerEpoch * 50 > len(train_generator):
    print("Adjusting steps_per_epoch to fit available data.")
    stepsPerEpoch = len(train_generator) // 50

if validationSteps * 50 > len(validation_generator):
    print("Adjusting validation_steps to fit available data.")
    validationSteps = len(validation_generator) // 50

# Fit the model for 50 epochs
history = model.fit(
    train_generator,                    # Training data generator
    validation_data=validation_generator, # Validation data generator
    epochs=50,                          # Total number of epochs
    steps_per_epoch=stepsPerEpoch,      # Steps per epoch for training
    validation_steps=validationSteps,   # Steps per epoch for validation
    callbacks=callbacks_list,           # List of callbacks
    verbose=1                           # Verbosity mode
)


In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# Standard data science libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics import classification_report

# Set paths
dataDirectory = "/kaggle/input/gastric"
train_path = dataDirectory + '/train'
test_path = dataDirectory + '/test'

# Custom normalization function
def custom_preprocess(x):
    # Normalizing the image pixel values to a range of 0 to 1
    x = x / 255.0
    return preprocess_input(x)  # Apply ResNet50's preprocessing after custom normalization

# ImageDataGenerator with enhanced data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    validation_split=0.2,  # Set validation split
    preprocessing_function=custom_preprocess  # Use custom normalization followed by ResNet50 preprocessing
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocess,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocess
)

# Load data
selectedClasses = ['0_normal', '2_polyps', '3_esophagitis', '1_ulcerative_colitis']
batchSize = 42

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='training'
)

validation_generator = validation_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    classes=selectedClasses,
    shuffle=False,
    batch_size=batchSize
)

# ResNet50 Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Initially, freeze all layers

# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)  # Add batch normalization for better performance
x = Dropout(0.5)(x)  # Regularization via dropout
x = Dense(512, activation='relu')(x)  # Dense layer for better feature learning
x = Dropout(0.5)(x)  # Dropout again for more regularization
predictions = Dense(len(selectedClasses), activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze the last few layers of ResNet50 for fine-tuning
for layer in base_model.layers[-20:]:  # Fine-tune the last 20 layers of ResNet50
    layer.trainable = True

# Compile the model
initial_learning_rate = 0.0001
optimizer = Adam(learning_rate=initial_learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Callbacks for training
checkpoint = ModelCheckpoint(
    'best_model.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min'
)

early_stopping = EarlyStopping(
    monitor='val_loss', patience=10, verbose=1, restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batchSize,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batchSize,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc*100:.2f}%')

# Predict on test set and generate classification report
test_generator.reset()
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

print("Classification Report:")
print(classification_report(test_generator.classes, predicted_classes, target_names=selectedClasses))

# Plot training & validation accuracy & loss
plt.figure(figsize=(12, 6))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
